# Processing Big Data - Data Ingestion
© Explore Data Science Academy

## Honour Code
I **KUSASALETHU**, **SITHOLE**, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).
    Non-compliance with the honour code constitutes a material breach of contract.



## Context 

To work constructively with any dataset, one needs to create an ingestion profile to make sure that the data at the source can be readily consumed. For this section of the predict, as the Data Engineer in the team, you will be required to design and implement the ingestion process. For the purposes of the project the AWS cloud storage service, namely, the S3 bucket service will act as your data source. All the data required can be found [here](https://processing-big-data-predict-stocks-data.s3.eu-west-1.amazonaws.com/stocks.zip).

<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/DataIngestion.jpg"
     alt="Data Ingestion"
     style="float: center; padding-bottom=0.5em"
     width=40%/>
     <p><em>Figure 1. Data Ingestion</em></p>
</div>

Your manager, Gnissecorp Atadgib, knowing very well that you've recently completed your Data Engineering qualification, asks you to make use of Apache Spark for the ingestion as well as the rest of the project. His rationale being, that stock market data is generated every day and is quite time-sensitive and would require scalability when deploying to a production environment. 

## Dataset - US Nasdaq




<div align="center" style="width: 600px; font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/data_engineering/transform/predict/Nasdaq.png"
     alt="Nasdaq"
     style="float: center; padding-bottom=0.5em"
     width=50%/>
     <p><em>Figure 2. Nasdaq</em></p>
</div>

The data that you will be working with is a historical snapshot of market data taken from the Nasdaq electronic market. This dataset contains historical daily prices for all tickers currently trading on Nasdaq. The up-to-date list can be found on their [website](https://www.nasdaq.com/)


The provided data contains price data dating back from 02 January 1962 up until 01 April 2020. The data found in the S3 bucket has been stored in the following structure:

```
     stocks/<Year>/<Month>/<Day>/stocks.csv
```
Each CSV file for every trading day contains the following details:
- **Date** - specifies trading date
- **Open** - opening price
- **High** - maximum price during the day
- **Low** - minimum price during the day
- **Close** - close price adjusted for splits
- **Adj Close** - close price adjusted for both dividends and splits
- **Volume** - the number of shares that changed hands during a given day

## Basic initialisation
To get you started, let's import some basic Python libraries as well as Spark modules and functions.

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *

Remember that we need a `SparkContext` and `SparkSession` to interface with Spark.
We will mostly be using the `SparkContext` to interact with RDDs and the `SparkSession` to interface with Python objects.

> ℹ️ **Instructions** ℹ️
>
>Initialise a new **Spark Context** and **Session** that you will use to interface with Spark.

In [80]:
#Initialise Spark Context and Session
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

## Investigate dataset schema
At this point, it is enough to read in a single file to ascertain the data structure. You will be required to use the information obtained from the small subset to create a data schema. This data schema will be used when reading the entire dataset using Spark.

> ℹ️ **Instructions** ℹ️
>
>Make use of Pandas to read in a single file and investigate the plausible data types to be used when creating a Spark data schema. 
>
>*You may use as many coding cells as necessary.*

In [81]:
! cd

C:\Users\skusa\DevWork\course\Sprints\Processing Big Data\processing-big-data-predict\Task1_data_ingestion


In [82]:
#Read one stock price csv as pandas
pd_df = pd.read_csv(r"..\data\stocks\stocks\1962\01\02\stocks.csv")

pd_df.head()

,Date,Open,High,Low,Close,Adj Close,Volume,stock
0,1962-01-02,6.532155,6.556185,6.532155,6.532155,1.536658,55900.0,AA
1,1962-01-02,6.125844,6.160982,6.125844,6.125844,1.414651,59700.0,ARNC
2,1962-01-02,0.837449,0.837449,0.823045,0.823045,0.145748,352200.0,BA
3,1962-01-02,1.604167,1.619792,1.588542,1.604167,0.136957,163200.0,CAT
4,1962-01-02,0.000000,3.296131,3.244048,3.296131,0.051993,105600.0,CVX


In [83]:
pd_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       20 non-null     object 
 1   Open       20 non-null     float64
 2   High       20 non-null     float64
 3   Low        20 non-null     float64
 4   Close      20 non-null     float64
 5   Adj Close  20 non-null     float64
 6   Volume     20 non-null     float64
 7   stock      20 non-null     object 
dtypes: float64(6), object(2)
memory usage: 1.4+ KB


## Read CSV files

When working with big data, it is often not tenable to keep processing an entire data batch when you are in the process of development - this can be quite time-consuming. If the data is uniform, it is sufficient to work with a smaller subset to create basic functionality. Your manager has identified the year **1962** to perform the initial testing for data ingestion. 

> ℹ️ **Instructions** ℹ️
>
>Read in the data for **1962** using a data schema that purely uses string data types. You will be required to convert to the appropriate data types at a later stage.
>
>*You may use as many coding cells as necessary.*

In [84]:
#Read all data for 1962 into Spark dataframe
df = spark.read.option("inferSchema", "false").csv(r"..\data\stocks\stocks\1962\*\*\stocks.csv", header=True)
df.show(10)

+----------+------------------+------------------+------------------+------------------+------------------+---------+-----+
|      Date|              Open|              High|               Low|             Close|         Adj Close|   Volume|stock|
+----------+------------------+------------------+------------------+------------------+------------------+---------+-----+
|1962-02-19| 5.839290142059326| 5.907374858856201| 5.839290142059326| 5.863319873809815|1.3863292932510376|  29900.0|   AA|
|1962-02-19| 5.481634140014648|5.5284857749938965| 5.481634140014648| 5.516772747039795|1.2804527282714844|  32000.0| ARNC|
|1962-02-19|0.9074074029922484|0.9156378507614136|0.8991769552230835| 0.903292179107666|0.1614154428243637| 619400.0|   BA|
|1962-02-19|1.6770833730697632|1.6927083730697632|1.6614583730697632|1.6770833730697632|0.1440587043762207| 170400.0|  CAT|
|1962-02-19|               0.0|3.5788691043853764|              20.0| 3.549107074737549|0.0565012246370315| 273600.0|  CVX|
|1962-02

In [85]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- stock: string (nullable = true)



In [86]:
df.describe().show()

+-------+----------+------------------+------------------+--------------------+--------------------+--------------------+-----------------+-----+
|summary|      Date|              Open|              High|                 Low|               Close|           Adj Close|           Volume|stock|
+-------+----------+------------------+------------------+--------------------+--------------------+--------------------+-----------------+-----+
|  count|      5106|              5106|              5106|                5084|                5106|                5106|             5085| 5106|
|   mean|      NULL|1.0904873526012002|16.757624946793637|  15.728619917198033|   16.64199179044607|   5.986642513535306|540930.2458210423| NULL|
| stddev|      NULL|2.3644535253049095| 53.91407348193544|     51.322922898144|  53.546771399008016|   24.64637054715319|864596.2440525509| NULL|
|    min|1962-01-02|               0.0|               0.0|-0.04043552317256...|0.053624976426362984|-0.04100876298375...|   

In [87]:
df.where(F.isnotnull(F.col('Low'))).orderBy(F.col('Low')).show(10)

+----------+----+-------+--------------------+-------+------------------+------+-----+
|      Date|Open|   High|                 Low|  Close|         Adj Close|Volume|stock|
+----------+----+-------+--------------------+-------+------------------+------+-----+
|1962-12-14| 0.0|239.375|-0.04043552317256...|239.375|112.29613494873048| 400.0|  NAV|
|1962-12-11| 0.0|244.375|-0.19642100914970...|244.375|114.64173126220705| 400.0|  NAV|
|1962-12-04| 0.0|249.375|-0.20796873271593...|249.375|115.56932830810548| 500.0|  NAV|
|1962-12-05| 0.0| 248.75|-0.41136139125085...| 248.75|115.27965545654295| 500.0|  NAV|
|1962-12-31| 0.0|250.625|-0.44164770838965...| 248.75| 116.6941375732422| 600.0|  NAV|
|1962-12-26| 0.0|246.875|-0.83186249301530...| 246.25|115.52132415771484| 500.0|  NAV|
|1962-12-18| 0.0| 238.75|-0.91096969052325...| 236.25| 110.8301010131836| 900.0|  NAV|
|1962-12-03| 0.0|  247.5|-1.54637213084751...|  247.5|114.70040130615234| 500.0|  NAV|
|1962-12-17| 0.0|240.625|-1.67761155076315.

In [88]:
df.where(F.isnotnull(F.col('Adj Close'))).orderBy(F.col('Adj Close')).show(10)

+----------+----+------------------+------------------+------------------+--------------------+---------+-----+
|      Date|Open|              High|               Low|             Close|           Adj Close|   Volume|stock|
+----------+----+------------------+------------------+------------------+--------------------+---------+-----+
|1962-06-13| 0.0| 5.583333492279053| 5.541666507720947| 5.552083492279053|-0.04100876298375...|  54000.0|   FL|
|1962-06-05| 0.0| 5.666666507720947|           5.59375| 5.614583492279053|-0.16605921310473...|  52800.0|   FL|
|1962-06-18| 0.0| 5.666666507720947|               5.5|            5.5625|-0.32137937514811...|  42000.0|   FL|
|1962-06-22| 0.0| 5.364583492279053| 5.291666507720947| 5.291666507720947|-0.50455847278181...|  67200.0|   FL|
|1962-06-14| 0.0| 5.583333492279053| 5.520833492279053| 5.520833492279053|-0.87224455509039...|  48000.0|   FL|
|1962-06-12| 0.0| 5.666666507720947| 5.604166507720947| 5.604166507720947|-1.07113589056835...|  50400.0

## Update column names
To make the data easier to work with, you will need to make a few changes:
1. Column headers should all be in lowercase; and
2. Whitespaces should be replaced with underscores.


> ℹ️ **Instructions** ℹ️
>
>Make sure that the column headers are all in lowercase and that any whitespaces are replaced with underscores.
>
>*You may use as many coding cells as necessary.*

In [89]:
# Updating column names

# function for renaming columns
def column_rename(df):
    for col in df.columns:
        df = df.withColumnRenamed(col, col.lower().replace(" ", "_"))
    return df

# Apply function
renamed_df = column_rename(df)

renamed_df.show()

+----------+------------------+------------------+------------------+------------------+--------------------+---------+-----+
|      date|              open|              high|               low|             close|           adj_close|   volume|stock|
+----------+------------------+------------------+------------------+------------------+--------------------+---------+-----+
|1962-02-19| 5.839290142059326| 5.907374858856201| 5.839290142059326| 5.863319873809815|  1.3863292932510376|  29900.0|   AA|
|1962-02-19| 5.481634140014648|5.5284857749938965| 5.481634140014648| 5.516772747039795|  1.2804527282714844|  32000.0| ARNC|
|1962-02-19|0.9074074029922484|0.9156378507614136|0.8991769552230835| 0.903292179107666|  0.1614154428243637| 619400.0|   BA|
|1962-02-19|1.6770833730697632|1.6927083730697632|1.6614583730697632|1.6770833730697632|  0.1440587043762207| 170400.0|  CAT|
|1962-02-19|               0.0|3.5788691043853764|              20.0| 3.549107074737549|  0.0565012246370315| 273600.0

## Null Values
Null values often represent missing pieces of data. It is always good to know where your null values lie - so you can quickly identify and remedy any issues stemming from these.

> ℹ️ **Instructions** ℹ️
>
>Write code to count the number of null values found in each column.
>
>*You may use as many coding cells as necessary.*

In [90]:
#Count null values per column
renamed_df.select([F.sum(F.col(column).isNull().cast("int")).alias(column) for column in renamed_df.columns]).show()

+----+----+----+---+-----+---------+------+-----+
|date|open|high|low|close|adj_close|volume|stock|
+----+----+----+---+-----+---------+------+-----+
|   0|   0|   0| 22|    0|        0|    21|    0|
+----+----+----+---+-----+---------+------+-----+



## Data type conversion - The final data schema

Now that we have identified the number of missing values in the data set, we'll move on to convert our data schema to the required data types. 

> ℹ️ **Instructions** ℹ️
>
>Use typecasting to convert the string data types in your current data schema to more appropriate data types.
>
>*You may use as many coding cells as necessary.*

In [91]:
#Typecasting columns
typecasted_df = renamed_df.withColumn("date", F.col("date").cast(TimestampType())) \
       .withColumn("open", F.col("open").cast(FloatType())) \
       .withColumn("high", F.col("high").cast(FloatType())) \
       .withColumn("low", F.col("low").cast(FloatType())) \
       .withColumn("close", F.col("close").cast(FloatType())) \
       .withColumn("adj_close", F.col("adj_close").cast(FloatType())) \
       .withColumn("volume", F.col("volume").cast(FloatType()))

typecasted_df.printSchema()

root
 |-- date: timestamp (nullable = true)
 |-- open: float (nullable = true)
 |-- high: float (nullable = true)
 |-- low: float (nullable = true)
 |-- close: float (nullable = true)
 |-- adj_close: float (nullable = true)
 |-- volume: float (nullable = true)
 |-- stock: string (nullable = true)



## Consolidate missing values
We have to check if the data type conversion above was done correctly.
If the casting was not successful, a null value gets inserted into the dataframe. You can thus check for successful conversion by determining if any null values are included in the resulting dataframe.


> ℹ️ **Instructions** ℹ️
>
>Write code to compare the number of invalid entries (nulls) pre-conversion and post-conversion.
>
>*You may use as many coding cells as necessary.*

In [92]:
# Calculate null counts before and after typecasting
null_count_before = renamed_df.select([F.sum(F.col(column).isNull().cast("int")).alias(column) for column in renamed_df.columns])
null_count_after = typecasted_df.select([F.sum(F.col(column).isNull().cast("int")).alias(column) for column in typecasted_df.columns])

In [93]:
# Rename columns in null_count_after to ensure unique names
null_count_after = null_count_after.select([F.col(column).alias(f"{column}_after") for column in null_count_after.columns])

In [94]:
# Add a dummy column to facilitate the join
null_count_before = null_count_before.withColumn("dummy_key", F.lit(1))
null_count_after = null_count_after.withColumn("dummy_key", F.lit(1))

# Join the DataFrames on the dummy column
joined_df = null_count_before.join(null_count_after, "dummy_key")

In [95]:
# Subtract the columns to get the difference in null counts
for column in renamed_df.columns:
    joined_df = joined_df.withColumn(column, F.col(f"{column}_after") - F.col(column))

# Drop the dummy column and the original null count columns
result_df = joined_df.drop("dummy_key").drop("dummy_key_after").drop(*[f"{column}_after" for column in renamed_df.columns])

# Show the result
result_df.show()

+----+----+----+---+-----+---------+------+-----+
|date|open|high|low|close|adj_close|volume|stock|
+----+----+----+---+-----+---------+------+-----+
|   0|   0|   0| 20|    0|       21|     0|    0|
+----+----+----+---+-----+---------+------+-----+



Here you should be able to see if any of your casts went wrong. 
Do not attempt to correct any missing values at this point. This will be dealt with in later sections of the predict.

## Generate parquet files
When writing in Spark, we typically use parquet format. This format allows parallel writing using Spark's optimisation while maintaining other useful things like metadata.

When writing, it is good to make sure that the data is sufficiently partitioned. 

Generally, data should be partitioned with one partition for every 200MB of data, but this also depends on the size of your cluster and executors. 


### Check the size of the dataframe before partitioning

In [96]:
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer

In [97]:
rdd = df.rdd._reserialize(AutoBatchedSerializer(PickleSerializer()))
obj = rdd.ctx._jvm.org.apache.spark.mllib.api.python.SerDe.pythonToJava(rdd._jrdd, True)
size = sc._jvm.org.apache.spark.util.SizeEstimator.estimate(obj)
size_MB = size/1000000
partitions = max(int(size_MB/200), 2)
print(f'The dataframe is {size_MB} MB')

The dataframe is 6.963128 MB


### Write parquet files to the local directory
> ℹ️ **Instructions** ℹ️
>
> Use the **coalesce** function and the number of **partitions** derived above to write parquet files to your local directory 
>
>*You may use as many coding cells as necessary.*

In [102]:
#repartition dataframe and write to parquet
coalesced_df = typecasted_df.coalesce(partitions)

coalesced_df.write.parquet(r"..\data\processed_data\1962\ingested_data")